# DATA CLEANING
In this notebook, we are going to perform some techniques on the gathered data, in order improve its quality for the future development of other tools.

In [2]:
import pandas as pd
import json
import numpy as np

import datetime
from datetime import timedelta

import statistics
import math

from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# We load the dataset with the preprocessed arrival_times
buses_data_at = pd.read_csv('../Data/Raw/buses_data.csv',
    dtype={
        'line': 'str',
        'destination': 'str',
        'stop': 'uint16',
        'bus': 'uint16',
        'day_trip': 'uint16',
        'given_coords': 'bool',
        'pos_in_burst':'uint16',
        'estimateArrive': 'int32',
        'DistanceBus': 'int32',
        'request_time': 'int32',
        'lat':'float32',
        'lon':'float32'
    }
)[['line','destination','stop','bus','datetime','estimateArrive','DistanceBus','given_coords','lat','lon']]

#Parse the dates
buses_data_at['datetime'] = pd.to_datetime(buses_data_at['datetime'], format='%Y-%m-%d %H:%M:%S.%f')
buses_data_at.head()

,line,destination,stop,bus,datetime,estimateArrive,DistanceBus,given_coords,lat,lon
0,82,PITIS,1686,4717,2020-02-25 20:41:06.748810,87,644,True,40.464340,-3.724854
1,132,HOSPITAL LA PAZ,1686,4836,2020-02-25 20:41:06.748810,88,694,True,40.463127,-3.713448
2,G,CIUDAD UNIVERSITARIA,1686,8618,2020-02-25 20:41:06.748810,256,1117,True,40.434059,-3.718819
3,132,HOSPITAL LA PAZ,1686,4716,2020-02-25 20:41:06.748810,535,1867,True,40.465935,-3.718888
4,82,PITIS,1686,4718,2020-02-25 20:41:06.748810,605,1939,True,40.473091,-3.735647


In [6]:
buses_data_at.loc[buses_data_at.estimateArrive > 70000].iloc[0]

line                                       1
destination                       CRISTO REY
stop                                     273
bus                                     4680
datetime          2020-02-25 20:45:37.566339
estimateArrive                        999999
DistanceBus                             3866
given_coords                           False
lat                                  40.4201
lon                                 -3.70197
Name: 3482, dtype: object

In [7]:
buses_data_at.loc[buses_data_at.DistanceBus < 0].iloc[0]

line                                       1
destination                      PROSPERIDAD
stop                                    4514
bus                                     8324
datetime          2020-02-25 20:41:08.933044
estimateArrive                           230
DistanceBus                              -12
given_coords                            True
lat                                  40.4445
lon                                 -3.68462
Name: 505, dtype: object

In [3]:
#Load line_stops_dict
with open('M6Data/lines_collected_dict.json', 'r') as f:
    lines_collected_dict = json.load(f)

In [4]:
#Route and lines shapes
lines_shapes = pd.read_csv('M6Data/lines_shapes.csv')

In [5]:
#Stops
stops = pd.read_csv('M6Data/stops.csv')

In [6]:
#FUNCTIONS
def haversine(coord1, coord2):
    '''
    Returns distance between two given coordinates in meters
    
        Parameters
        ----------
        coord1 : tuple
            Coordinates of the first point
        coord2 : tuple
            Coordinates of the second point
            
    '''
    
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    
    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

def calculate_coords(df,stop_id,dist_to_stop) :
    '''
    Returns the calculated coordinates of the bus
    
    Parameters
        ----------
        df : dataframe
            Dataframe where we want to find the calculated coords
        stop : str
        dist_to_stop : int
            Distance to the stop in meters
    '''
    line_sn = df.iloc[0].line_sn
    direction = str(df.iloc[0].direction)
    bus_distance = int(lines_collected_dict[line_sn][direction]['distances'][str(stop_id)]) - dist_to_stop
    nearest_row = find_nearest_row_by_dist(df,bus_distance)
    return nearest_row.lon, nearest_row.lat

def find_nearest_row_by_dist(df,dist_traveled) :
    """
    Returns the row nearest to the distance traveled passed in the dataframe
    
        Parameters
        ----------
        df : dataframe
            Dataframe where we want to find the row
        dist_traveled : int
            Distance to origin of the line in meters
    """
    min_dist_error = 1000000.0
    df_reduced = df.loc[(df.dist_traveled>dist_traveled-100)&(df.dist_traveled<dist_traveled+100)]
    if df_reduced.shape[0]!=0:
        for row in df_reduced.itertuples() :
            error = abs(row.dist_traveled-dist_traveled)
            if  error < min_dist_error :
                min_dist_error = error
                nearest_row = row
    else :
        nearest_row = df.iloc[0]
    return nearest_row

## Check that the lines, destinations and stops are coherent
First, we are going to check that the group of values line destination and stop are coherent with the destinations and stop defined for each line, if there is a non-coherent row, we drop it.


In [7]:
def check_line_destination_stop(row):
    '''
    Returns true if the values of line, destination and stop are coherent. False if they arent
    '''
    if row.line in lines_collected_dict.keys() :
        if row.destination in lines_collected_dict[row.line]['destinations'] :
            direction = '1' if row.destination == lines_collected_dict[row.line]['destinations'][1] else '2'
            if str(row.stop) in lines_collected_dict[row.line][direction]['stops'] :
                return True
    return False


line_destination_stop_cond = buses_data_at[['line','destination','stop']].parallel_apply(check_line_destination_stop,axis=1)

In [8]:
#Number of rows that meet the condition
line_destination_stop_cond.value_counts()

True    5956293
dtype: int64

## Values of DistanceBus higher than the line length or negative
Now, we are going to eliminate the rows which distance remaining to the stop is higher than the line length. And also the one that have a negative value.


In [9]:
def check_distance_remaining (row) :
    '''
    Returns true if the distance remaining is positive and smaller than the line lenght
    '''
    direction = '1' if row.destination == lines_collected_dict[row.line]['destinations'][1] else '2'
    return (row.DistanceBus < int(lines_collected_dict[row.line][direction]['length'])) and (row.DistanceBus >= 0)
    
distance_remaining_cond = buses_data_at[['line','destination','DistanceBus']].parallel_apply(check_distance_remaining,axis=1)

In [10]:
#Number of rows that meet the condition
distance_remaining_cond.value_counts()

True     5825568
False     130725
dtype: int64

## Values of estimateArrive higher than the time it takes to go through the line lenght at an speed of 2m/s,instantaneous speed is over 120 km/h and negative values.

If the time remaining for the bus to arrive the stop is higher than the time it will take to go through the hole line at a speed of 2 meters per second, we drop that columns.
Also, if the DistanceBus divided by the estimateArrive returns a speed higher than 150 km/h or the time remaining is negative.

In [11]:
def check_time_remaining (row) :
    '''
    Returns true if the estimateArrive time meets the conditions above
    '''
    direction = '1' if row.destination == lines_collected_dict[row.line]['destinations'][1] else '2'
    return (row.estimateArrive > 0) and \
        (row.estimateArrive < (int(lines_collected_dict[row.line][direction]['length'])/2)) and \
        (3.6*row.DistanceBus/row.estimateArrive) < 120 

estimate_arrive_cond = buses_data_at[['line','destination','estimateArrive','DistanceBus']].parallel_apply(check_time_remaining,axis=1)

In [12]:
#Number of rows that meet the condition
estimate_arrive_cond.value_counts()

True     5733995
False     222298
dtype: int64

## Values whose difference between the estimated time of arrival and ''datetime''+''estimateArrive'' is bigger than 10 minutes.
This can be caused by accidents or other rare sucesses, so we shouldnt consider this a normal behaviour.

In [13]:
def check_arrival_estimation_error (row) :
    '''
    Returns true if the error in the arrival time estimation is lower than 10 minutes.
    '''
    seconds_error = abs(((row.datetime + timedelta(seconds=int(row.estimateArrive))) - row.arrival_time).total_seconds())
    return seconds_error < 600 

arrival_estim_error_cond = buses_data_at[['datetime','estimateArrive','arrival_time']].parallel_apply(check_arrival_estimation_error,axis=1)

In [14]:
#Number of rows that meet the condition
arrival_estim_error_cond.value_counts()

True     5800027
False     156266
dtype: int64

## Check that the distance between given and calculated coordinates is smaller than 300 meters. 
Select rows with a coherent value of the given latitudes and longitudes.

In [15]:
def check_given_coordinates (row) :
    '''
    Returns true if the coordinates given value is coherent. False if not or if the coordinates given are [0,0].
    '''
    distance = 1000
    if row.given_coords :
        direction = '1' if row.destination == lines_collected_dict[row.line]['destinations'][1] else '2'
        line_id = lines_collected_dict[row.line]['line_id']
        line = lines_shapes.loc[(lines_shapes.line_id == int(line_id))&(lines_shapes.direction == int(direction))] 
        calc_lon,calc_lat = calculate_coords(line,row.stop,row.DistanceBus)
        distance = haversine((calc_lat,calc_lon),(row.lat,row.lon))
    
    return distance < 300  

given_coordinates_cond = buses_data_at[['line','destination','stop','DistanceBus','given_coords','lat','lon']].parallel_apply(check_given_coordinates,axis=1)

In [16]:
given_coordinates_cond.value_counts()

False    5326436
True      629857
dtype: int64